In [130]:
"""
Live Project Deliverable 2.2 
For each story and their corresponding summaries

    Calculate the ROUGE score between each sentence of the story and each of the summaries corresponding to the story
    Grab the top five sentences from the story with respect to their ROUGE F1-score and return them as a list
    Return the ROUGE F1-score of those top five sentences as a list

"""

import string 
import os
import rouge
from rouge_score import rouge_scorer
import pickle
#import pdb # For Debugging

In [131]:
# Working directories 
workingDirectory = '/home/richard/Desktop/Python Live Project/test'
cleanDirectory = workingDirectory+'/cleaned/'

# Make directory for cleaned data
os.chdir(workingDirectory)
if os.path.isdir(cleanDirectory):
    pass
else:
    os.mkdir(cleanDirectory)
    
os.chdir(cleanDirectory)    


In [132]:
# ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=False)

# Open Master List of formatted stories with sentences and summaries 
with open ('dict.pkl', 'rb') as fp:
    itemlist = pickle.load(fp)
    
    # Add a Dictionary to the Master List for each story for the ROUGE scores
    # Iterate through the master list
    for story in itemlist:
        # Dictionary to contain Rouge scores and index of sentence 
        rougeList = {}
        # Add to story Dictionary 
        story["ROUGE"] = rougeList
        story["BestFive"] = []

        for summary in story["summaries"]:
            #print("Summary:  ",summary) 
            for sentence in story["news"]:
                # Calculate ROUGE score
                scores = scorer.score(summary, sentence)
                sentenceScore = scores["rouge1"][2]  # the F score
                sentenceIndex = story["news"].index(sentence)
                # Add these to the ROUGE dictionary
                # If dictionary is empty or 
                # If dictionary key pair not present    
                if not sentenceIndex in rougeList:
                    rougeList[sentenceIndex] = sentenceScore
                #  Otherwise if current sentence score is higher than previous summary/news comparison
                # Save higher score
                if rougeList[sentenceIndex] < sentenceScore:
                    rougeList[sentenceIndex] = sentenceScore

        # Extract top 5 sentences from stories
        # use dictionary sorted method
        bestFive = sorted(rougeList, key=rougeList.get, reverse=True)[:5]
        
        # Create a list to add to the master list 
        bestFiveList = []
        for index in bestFive:
            bestFiveList.append(story["news"][index])
            BFstring = ("News sentence no: ",index, ":", "ROUGE Score: ", rougeList.get(index))
            bestFiveList.append(BFstring)
        story["BestFive"] = bestFiveList

In [133]:
with open(cleanDirectory + 'dictrouge.pkl', 'wb') as fp:
    pickle.dump(itemlist, fp)

In [134]:
import pandas as pd

In [135]:
stories = pd.DataFrame(itemlist) 

In [138]:
with open(cleanDirectory + 'dictDataFrame.pkl', 'wb') as fp:
    pickle.dump(stories, fp)